In [1]:
#BPR
import numpy
import tensorflow as tf
import os
import random
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial
from tqdm import tqdm
%matplotlib inline

In [2]:
DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"
DATA_FOR_BPR = "data_for_BPR.csv"
SAVE_DIR = "../gamedata/train_result/"

In [ ]:
# #data preprocessing
# data = pd.read_csv(DATA_DIR+DATA_FILE)
# gameid = data['gameid'].drop_duplicates()
# gameid_list = []
# gameid_list = list(gameid)
# gameid2newid = {old:new+1 for new, old in enumerate(gameid_list)}
# data['new_game_id'] = data['gameid'].map(gameid2newid)

# events = data['eventname'].drop_duplicates()
# events_list = list(events)
# events2id = {old:new+1 for new,old in enumerate(events_list)}
# data['new_event_id'] = data['eventname'].map(events2id)

In [ ]:
#data.to_csv(DATA_DIR+'data_for_BPR.csv',index=False)

In [ ]:
#data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [ ]:
#data[data['new_game_id']==63]

In [3]:
def load_data(data_path):
    game_events = defaultdict(set)
    game_count = -1
    event_count = -1
    with open(data_path,'r') as f:
        header_line = next(f)
        for line in f.readlines():
            _, _, _, _, _, _, gameid, eventid = line.split(",")
            gameid = int(gameid)
            eventid = int(eventid)
            #print(u,i)
            game_events[gameid].add(eventid)
            game_count = max(gameid, game_count)
            event_count = max(eventid, event_count)
    print ("game_count:", game_count)
    print ("event_count:", event_count)
    #return max_u_id, max_i_id, user_ratings
    return game_count,event_count,game_events
    

#data_path = os.path.join('D:\\tmp\\ml-100k', 'u.data')
#user_count, item_count, user_ratings = load_data(data_path)
game_count,event_count,game_events_dict = load_data(DATA_DIR+DATA_FOR_BPR)

game_count: 1108
event_count: 11189


In [4]:
#
def generate_test(game_events_dict):
    user_test = dict()
    for u, i_list in game_events_dict.items():
        user_test[u] = random.sample(game_events_dict[u], 1)[0]
    return user_test

game_test_set = generate_test(game_events_dict)

In [15]:
#generate dataset for training
def generate_train_batch(game_events_dict, game_test_set, event_count, batch_size=128):
    t = []
    
    #print('generate batch..')
    for b in range(batch_size):
        break_flag1 = 0
        break_flag2 = 0
        u = random.sample(game_events_dict.keys(), 1)[0]
        #print('u:',u)
        i = random.sample(game_events_dict[u], 1)[0]
        while i == game_test_set[u]:
            break_flag1 += 1
            i = random.sample(game_events_dict[u], 1)[0]
            if break_flag1 == 20:
                #game_events_dict.pop('u',None)
                #game_test_set.pop('u',None)
                break
            #print('i:',i)
        j = random.randint(1, event_count)
        while j in game_events_dict[u]:
            break_flag2 += 1
            j = random.randint(1, event_count)
            if break_flag2 == 20:
                #game_events_dict.pop('u',None)
                #game_test_set.pop('u',None)
                break
            #print('j:',j)
        t.append([u, i, j])
        #print(u,i,j)
    #print('generate batch done!')
    return numpy.asarray(t)

#train_set = generate_train_batch(game_events_dict,game_test_set,event_count)

In [6]:
def generate_test_batch(game_events_dict, game_test_set, event_count):
    for u in game_events_dict.keys():
        t = []
        i = game_test_set[u]
        for j in range(1, event_count+1):
            if not (j in game_events_dict[u]):
                t.append([u, i, j])
        yield numpy.asarray(t)

In [18]:
def bpr_mf(game_count, event_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [game_count+1, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [event_count+1, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
    
    # MF predict: u_i > u_j
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keepdims=True)
    
    # AUC for one user:
    # reasonable iff all (u,i,j) pairs are from the same user
    # 
    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    
    l2_norm = tf.add_n([
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

In [23]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(game_count, event_count, 20)
#     print(u, i, j, mf_auc, bprloss, train_op)
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    print('start epoch')
    for epoch in range(1, 2):
        print ("epoch: ", epoch)
        _batch_bprloss = 0
        for k in tqdm(range(1, 5000)): # uniform samples from training set
            uij = generate_train_batch(game_events_dict, game_test_set,event_count)

            _bprloss, _train_op = session.run([bprloss, train_op], 
                                feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        
        print ("bpr_loss: ", _batch_bprloss / k)
        print ("_train_op")

        games_count = 0
        _auc_sum = 0.0
        test_bprloss = 0.0
        # each batch will return only one user's auc
        for t_uij in tqdm(generate_test_batch(game_events_dict, game_test_set, event_count)):

            _auc, _test_bprloss = session.run([mf_auc, bprloss],
                                    feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]}
                                )
            games_count += 1
            _auc_sum += _auc
            test_bprloss += _test_bprloss
#            print ("test_loss: ", test_bprloss/games_count)
        print ("test_loss: ", test_bprloss/games_count, "test_auc: ", _auc_sum/games_count)
        print ("")
    variable_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variable_names)
    saver.save(session, SAVE_DIR)
    for k,v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)
print('all done')

  0%|          | 0/4999 [00:00<?, ?it/s]

Tensor("AddN:0", shape=(), dtype=float32) Tensor("Mean_2:0", shape=(), dtype=float32)
start epoch
epoch:  1


100%|██████████| 4999/4999 [00:24<00:00, 205.17it/s]
6it [00:00, 33.34it/s]

bpr_loss:  0.7004937100443847
_train_op


1108it [00:18, 60.32it/s]


test_loss:  1.3384761717775668 test_auc:  0.5112981665390922

Variable:  user_emb_w:0
Shape:  (1109, 20)
[[-0.07827675  0.03325282  0.03547279 ...  0.24047272  0.08952945
  -0.09412265]
 [-0.12227901  0.008841    0.14813305 ...  0.1654895   0.08843858
  -0.04588021]
 [-0.06300459  0.10307686  0.07544495 ... -0.12270406  0.0579744
  -0.07400693]
 ...
 [-0.09181725 -0.02721917  0.01927665 ... -0.03661722 -0.0572461
  -0.00751277]
 [-0.09103559  0.11260398 -0.08012318 ...  0.07623894 -0.04960154
  -0.02741719]
 [ 0.05854981 -0.00831366  0.16878232 ...  0.05671526  0.10351533
   0.02996822]]
Variable:  item_emb_w:0
Shape:  (11190, 20)
[[ 1.5270211e-02 -3.5029404e-02 -2.8490314e-02 ... -7.3622190e-03
  -1.4807060e-02 -1.5188259e-01]
 [ 5.5210702e-02  1.1759946e-01 -1.6139945e-02 ...  3.2923778e-03
   1.3766246e-04  5.4466449e-02]
 [-1.4415974e-02  7.5199150e-02 -6.2056758e-02 ...  1.6244659e-02
  -3.0633692e-02  1.0007460e-02]
 ...
 [ 6.3158825e-02 -3.8425114e-02 -1.6166653e-02 ...  2.45589

In [13]:
_test_bprloss

1.2801418

In [ ]:
saver.save(session, SAVE_DIR)

In [ ]:
session1 = tf.Session()
u1_dim = tf.expand_dims(values[0][0], 0)
u1_all = tf.matmul(u1_dim, values[1],transpose_b=True)
result_1 = session1.run(u1_all)
print (result_1)

In [ ]:
print("new recommedation ：")
p = numpy.squeeze(result_1)
p[numpy.argsort(p)[:-5]] = 0
for index in range(len(p)):
    if p[index] != 0:
        select = data[data['new_event_id']==index].drop_duplicates('eventname')
  
        print (index, p[index],select['eventname'])

In [ ]:
data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [ ]:
data[data['new_event_id']==11118]
data[data['new_game_id']==1].drop_duplicates('eventname')